<a href="https://colab.research.google.com/github/rawnakwow/Final_Lab-07_CSE366/blob/main/Caltech101_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install Required Libraries
!pip install torch torchvision grad-cam matplotlib scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39648 sha256=8a3febeec3b5a02d61489c689d169561d9662aa67d2e2ddaf87250ed85f8a520
  Stored in directory: /root/.cache/pip/wheels/8b/0d/d2/b12bec1ccc028921fb98158042ade2d19dae73925dfc636954
Successfully built grad-cam


In [ ]:
from google.colab import files
uploaded = files.upload()
file_name = "101_ObjectCategories.tar"

Saving image_0001.jpg to image_0001.jpg


In [ ]:
import tarfile

# Replace 'your_file.tar' with the name of your uploaded file
file_name = "101_ObjectCategories.tar"

# Open the tar file
with tarfile.open(file_name) as tar:
    tar.extractall()  # Extracts all files to the current directory
    tar.close()

print("Extraction completed.")


FileNotFoundError: [Errno 2] No such file or directory: '101_ObjectCategories.tar.gz'

In [ ]:
# Step 2: Import Libraries
# Import necessary libraries for deep learning, dataset handling, visualization, and evaluation.
import torch
import torch.nn as nn  # For defining neural network layers
import torch.optim as optim  # For optimization algorithms
from torchvision import datasets, transforms, models  # For datasets, image transformations, and pretrained models
from torch.utils.data import random_split, DataLoader  # For dataset splitting and batch data loading
from pytorch_grad_cam import GradCAM  # For Grad-CAM visualization
from pytorch_grad_cam.utils.image import show_cam_on_image  # For overlaying Grad-CAM on images
from sklearn.metrics import classification_report, confusion_matrix  # For model evaluation
import matplotlib.pyplot as plt  # For plotting
import numpy as np  # For numerical operations

In [ ]:
# Step 3: Set Up the Dataset
# Define transformations for the dataset to augment and preprocess the images
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to a fixed size
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(30),  # Randomly rotate images
    transforms.ColorJitter(brightness=0.4, contrast=0.4),  # Adjust brightness and contrast
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])



In [ ]:
# Load the dataset using ImageFolder
#  The actual path to the Caltech-101 dataset
dataset = datasets.ImageFolder(root=r"file names\airplanes\image_0001.jpg", transform=transform)



NameError: name 'datasets' is not defined

In [ ]:


# Split the dataset into training, validation, and test sets
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = int(0.1 * len(dataset))  # 10% for validation
test_size = len(dataset) - train_size - val_size  # Remaining 10% for testing
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for batch loading
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)  # Shuffle training data for randomness
val_loader = DataLoader(val_data, batch_size=32)  # No need to shuffle validation data
test_loader = DataLoader(test_data, batch_size=32)  # No need to shuffle test data


NameError: name 'dataset' is not defined

In [ ]:
# Step 4: Use a Pretrained Model
# Load a pretrained ResNet-50 model
model = models.resnet50(pretrained=True)
# Modify the final layer to match the number of classes in the Caltech-101 dataset (101 classes)
model.fc = nn.Linear(model.fc.in_features, 101)
# Move the model to GPU if available for faster training
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 108MB/s]


In [ ]:
# Step 5: Define Loss and Optimizer
# Define the loss function as CrossEntropyLoss for multi-class classification
criterion = nn.CrossEntropyLoss()
# Define the Adam optimizer with a learning rate of 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Step 6: Train the Model
# Train the model for a specified number of epochs
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0  # Track the total loss for the epoch
    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')  # Move data to GPU
        optimizer.zero_grad()  # Reset gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass (compute gradients)
        optimizer.step()  # Update model weights
        total_loss += loss.item()  # Accumulate the loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")  # Print average loss


NameError: name 'train_loader' is not defined

In [ ]:
# Step 7: Validate the Model
model.eval()  # Set model to evaluation mode
val_correct = 0
val_total = 0
with torch.no_grad():  # Disable gradient computation
    for images, labels in val_loader:
        images, labels = images.to('cuda'), labels.to('cuda')  # Move data to GPU
        outputs = model(images)  # Forward pass
        _, preds = torch.max(outputs, 1)  # Get predictions
        val_correct += (preds == labels).sum().item()  # Count correct predictions
        val_total += labels.size(0)  # Count total samples
print(f"Validation Accuracy: {val_correct / val_total:.2%}")  # Print validation accuracy


NameError: name 'val_loader' is not defined

In [ ]:
# Step 8: Evaluate on Test Data
test_correct = 0
test_total = 0
y_true, y_pred = [], []
with torch.no_grad():  # Disable gradient computation
    for images, labels in test_loader:
        images, labels = images.to('cuda'), labels.to('cuda')  # Move data to GPU
        outputs = model(images)  # Forward pass
        _, preds = torch.max(outputs, 1)  # Get predictions
        test_correct += (preds == labels).sum().item()  # Count correct predictions
        test_total += labels.size(0)  # Count total samples
        y_true.extend(labels.cpu().numpy())  # Store true labels
        y_pred.extend(preds.cpu().numpy())  # Store predicted labels
print(f"Test Accuracy: {test_correct / test_total:.2%}")  # Print test accuracy


NameError: name 'test_loader' is not defined

In [ ]:
# Step 9: Confusion Matrix and Classification Report
# Compute and display the confusion matrix and classification report
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=dataset.classes))


In [ ]:
# Step 10: Apply Grad-CAM
# Initialize Grad-CAM with the target layer
target_layer = model.layer4[-1]
cam = GradCAM(model=model, target_layer=target_layer, use_cuda=torch.cuda.is_available())

# Visualize Grad-CAM for the first 5 test images
for images, labels in test_loader:
    images = images.to('cuda')  # Move images to GPU
    grayscale_cam = cam(input_tensor=images)  # Generate Grad-CAM
    for i in range(min(5, len(images))):  # Visualize first 5 images
        # Overlay Grad-CAM on the original image
        cam_image = show_cam_on_image(images[i].cpu().permute(1, 2, 0).numpy(), grayscale_cam[i])
        plt.imshow(cam_image)  # Display the Grad-CAM visualization
        plt.show()
    break  # Stop after visualizing 5 images
